In [242]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Data Wrangling & Cleaning

In [243]:
transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='Transactions')
new_customer = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='NewCustomerList')
customer_demo = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerDemographic')
customer_add = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerAddress')

##### Transactions Dataset

In [244]:
transactions.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145


In [245]:
# Drop the first unnamed row
def titlehead(x):
    x.rename(columns=x.iloc[0],inplace=True)
    x.drop([0],inplace=True)

In [246]:
titlehead(transactions)

In [247]:
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
5,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [248]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 20000
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   transaction_id           20000 non-null  object
 1   product_id               20000 non-null  object
 2   customer_id              20000 non-null  object
 3   transaction_date         20000 non-null  object
 4   online_order             19640 non-null  object
 5   order_status             20000 non-null  object
 6   brand                    19803 non-null  object
 7   product_line             19803 non-null  object
 8   product_class            19803 non-null  object
 9   product_size             19803 non-null  object
 10  list_price               20000 non-null  object
 11  standard_cost            19803 non-null  object
 12  product_first_sold_date  19803 non-null  object
dtypes: object(13)
memory usage: 2.1+ MB


##### Features Engineering

In [249]:
count_missing_tran = transactions.isnull().sum()
percent_missing_tran = round(transactions.isnull().sum()/len(transactions)*100,1)
missing_transactions = pd.concat([count_missing_tran, percent_missing_tran],axis=1)
missing_transactions.columns = ['Missing(count)','Missing(%)']
missing_transactions

,Missing(count),Missing(%)
transaction_id,0,0.0
product_id,0,0.0
customer_id,0,0.0
transaction_date,0,0.0
online_order,360,1.8
order_status,0,0.0
brand,197,1.0
product_line,197,1.0
product_class,197,1.0
product_size,197,1.0


In [250]:
transactions['online_order'].value_counts()

True     9829
False    9811
Name: online_order, dtype: int64

In [251]:
# filling the online order column with the mode value
transactions['online_order'] = transactions['online_order'].fillna(transactions['online_order'].mode()[0])

In [252]:
# removing other data columns contain null values
transactions.dropna(inplace=True)

In [253]:
transactions.isnull().sum()

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

In [254]:
transactions.dtypes

transaction_id             object
product_id                 object
customer_id                object
transaction_date           object
online_order                 bool
order_status               object
brand                      object
product_line               object
product_class              object
product_size               object
list_price                 object
standard_cost              object
product_first_sold_date    object
dtype: object

In [255]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
transactions['standard_cost'] = pd.to_numeric(transactions['standard_cost'])
transactions['list_price'] = pd.to_numeric(transactions['list_price'])

In [256]:
transactions.dtypes

transaction_id                     object
product_id                         object
customer_id                        object
transaction_date           datetime64[ns]
online_order                         bool
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date            object
dtype: object

In [257]:
# product_first_sold_date column seems have a wrong date format
import datetime as dt
from datetime import datetime
def convert_to_datetime(num):
    dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + num - 2)
    return dt

In [258]:
transactions['product_first_sold_date']=transactions['product_first_sold_date'].apply(convert_to_datetime)
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [259]:
transactions['transaction_year'] = transactions['transaction_date'].dt.year
transactions['transaction_month'] = transactions['transaction_date'].dt.month
transactions['transaction_day'] = transactions['transaction_date'].dt.day

In [260]:
transactions.dtypes

transaction_id                     object
product_id                         object
customer_id                        object
transaction_date           datetime64[ns]
online_order                         bool
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date    datetime64[ns]
transaction_year                    int64
transaction_month                   int64
transaction_day                     int64
dtype: object

In [261]:
cat_col = [col for col in transactions.columns if transactions[col].dtype=='object']
cont_col = [col for col in transactions.columns if transactions[col].dtype!='object']

In [262]:
cat_col

['transaction_id',
 'product_id',
 'customer_id',
 'order_status',
 'brand',
 'product_line',
 'product_class',
 'product_size']

In [263]:
cont_col

['transaction_date',
 'online_order',
 'list_price',
 'standard_cost',
 'product_first_sold_date',
 'transaction_year',
 'transaction_month',
 'transaction_day']

##### New Customer List Dataset

In [264]:
new_customer.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.96,1.20,1.5000,1.2750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.84,0.84,1.0500,0.8925,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.09,1.09,1.0900,1.0900,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.60,0.75,0.7500,0.7500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.49,0.49,0.6125,0.6125,4,4,1.703125


In [265]:
new_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

In [266]:
# drop nan columns
cols = ['Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20']
new_customer = new_customer.drop(cols,axis=1)

##### Features Engineering

In [267]:
count_missing_new = new_customer.isnull().sum()
percent_missing_new = round(new_customer.isnull().sum()/len(new_customer)*100,1)
missing_new_customer = pd.concat([count_missing_new, percent_missing_new],axis=1)
missing_new_customer.columns = ['Missing(count)','Missing(%)']
missing_new_customer

,Missing(count),Missing(%)
first_name,0,0.0
last_name,29,2.9
gender,0,0.0
past_3_years_bike_related_purchases,0,0.0
DOB,17,1.7
job_title,106,10.6
job_industry_category,165,16.5
wealth_segment,0,0.0
deceased_indicator,0,0.0
owns_car,0,0.0


In [268]:
# filling columns with mode
new_customer['last_name'] = new_customer['last_name'].fillna(new_customer['last_name'].mode()[0])
new_customer['DOB'] = new_customer['DOB'].fillna(new_customer['DOB'].mode()[0])
new_customer['job_title'] = new_customer['job_title'].fillna(new_customer['job_title'].mode()[0])
new_customer['job_industry_category'] = new_customer['job_industry_category'].fillna(new_customer['job_industry_category'].mode()[0])

In [269]:
new_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            1000 non-null   object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  1000 non-null   datetime64[ns]
 5   job_title                            1000 non-null   object        
 6   job_industry_category                1000 non-null   object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

In [270]:
# checking for duplicated rows
new_customer[new_customer.duplicated()]

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value


In [271]:
cat_col_newcustomer=[col for col in new_customer if new_customer[col].dtype=='object']
cont_col_newcustomer=[col for col in new_customer if new_customer[col].dtype!='object']

In [272]:
cat_col_newcustomer

['first_name',
 'last_name',
 'gender',
 'job_title',
 'job_industry_category',
 'wealth_segment',
 'deceased_indicator',
 'owns_car',
 'address',
 'state',
 'country']

In [273]:
cont_col_newcustomer

['past_3_years_bike_related_purchases',
 'DOB',
 'tenure',
 'postcode',
 'property_valuation',
 'Rank',
 'Value']

In [274]:
new_customer['gender'].value_counts()

Female    513
Male      470
U          17
Name: gender, dtype: int64

In [275]:
new_customer['gender'].str.replace('U','Unspecified')

0        Male
1        Male
2      Female
3      Female
4      Female
        ...  
995      Male
996      Male
997    Female
998      Male
999      Male
Name: gender, Length: 1000, dtype: object

In [276]:
# This function converts DOB into age
def from_dob_to_age(born):
    today=dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [277]:
new_customer['Age'] = new_customer['DOB'].apply(lambda x:from_dob_to_age(x))

In [278]:
new_customer.drop('DOB',axis=1,inplace=True)

In [279]:
new_customer.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,Age
0,Chickie,Brister,Male,86,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750,64
1,Morly,Genery,Male,69,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750,51
2,Ardelis,Forrester,Female,10,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750,47
3,Lucine,Stutt,Female,64,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,42
4,Melinda,Hadlee,Female,34,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,56


##### CustomerDemographic Dataset

In [280]:
customer_demo.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7


In [281]:
titlehead(customer_demo)

In [282]:
customer_demo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [283]:
customer_demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1 to 4000
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   object
 1   first_name                           4000 non-null   object
 2   last_name                            3875 non-null   object
 3   gender                               4000 non-null   object
 4   past_3_years_bike_related_purchases  4000 non-null   object
 5   DOB                                  3913 non-null   object
 6   job_title                            3494 non-null   object
 7   job_industry_category                3344 non-null   object
 8   wealth_segment                       4000 non-null   object
 9   deceased_indicator                   4000 non-null   object
 10  default                              3698 non-null   object
 11  owns_car                             4000 n

##### Feature Engineering

In [284]:
count_missing_Customer_demographic = customer_demo.isnull().sum()
percent_missing_Customer_demographic = round(customer_demo.isnull().sum()/len(customer_demo) * 100, 1)
missing_customer_demo = pd.concat([count_missing_Customer_demographic, percent_missing_Customer_demographic], axis = 1)
missing_customer_demo.columns = ["Missing (count)", "Missing (%)"]
missing_customer_demo

,Missing (count),Missing (%)
customer_id,0,0.0
first_name,0,0.0
last_name,125,3.1
gender,0,0.0
past_3_years_bike_related_purchases,0,0.0
DOB,87,2.2
job_title,506,12.6
job_industry_category,656,16.4
wealth_segment,0,0.0
deceased_indicator,0,0.0


In [285]:
# filling columns with mode or mean
customer_demo['last_name'] = customer_demo['last_name'].fillna(customer_demo['last_name'].mode()[0])
customer_demo['DOB'] = customer_demo['DOB'].fillna(customer_demo['DOB'].mode()[0])
customer_demo['job_title'] = customer_demo['job_title'].fillna(customer_demo['job_title'].mode()[0])
customer_demo['job_industry_category'] = customer_demo['job_industry_category'].fillna(customer_demo['job_industry_category'].mode()[0])
customer_demo['tenure'] = customer_demo['tenure'].fillna(customer_demo['tenure'].mean())

In [286]:
# changing columns value types
customer_demo['past_3_years_bike_related_purchases']=pd.to_numeric(customer_demo['past_3_years_bike_related_purchases'])
customer_demo['tenure']=pd.to_numeric(customer_demo['tenure'])
customer_demo['DOB']=pd.to_datetime(customer_demo['DOB'])

In [287]:
customer_demo[customer_demo.duplicated()]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure


In [288]:
cat_col_customer_demo = [col for col in customer_demo.columns if customer_demo[col].dtype=='object']
cont_col_customer_demo = [col for col in customer_demo.columns if customer_demo[col].dtype!='object']

In [289]:
cat_col_customer_demo

['customer_id',
 'first_name',
 'last_name',
 'gender',
 'job_title',
 'job_industry_category',
 'wealth_segment',
 'deceased_indicator',
 'default',
 'owns_car']

In [290]:
cont_col_customer_demo

['past_3_years_bike_related_purchases', 'DOB', 'tenure']

In [291]:
customer_demo['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [292]:
customer_demo.replace({'gender':{'F':'Female','Femal':'Female','M':'Male','U':'Unspecified'}},inplace=True)

In [293]:
customer_demo.drop('default',axis=1,inplace=True)

In [294]:
customer_demo['Age']=customer_demo['DOB'].apply(lambda x:from_dob_to_age(x))

In [295]:
customer_demo.drop('DOB',axis=1,inplace=True)

In [296]:
customer_demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1 to 4000
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   object 
 1   first_name                           4000 non-null   object 
 2   last_name                            4000 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   job_title                            4000 non-null   object 
 6   job_industry_category                4000 non-null   object 
 7   wealth_segment                       4000 non-null   object 
 8   deceased_indicator                   4000 non-null   object 
 9   owns_car                             4000 non-null   object 
 10  tenure                               4000 non-null   float64
 11  Age                           

In [297]:
customer_demo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
1,1,Laraine,Medendorp,Female,93,Executive Secretary,Health,Mass Customer,N,Yes,11.0,68
2,2,Eli,Bockman,Male,81,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,41
3,3,Arlin,Dearle,Male,61,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,67
4,4,Talbot,Pristnor,Male,33,Business Systems Development Analyst,IT,Mass Customer,N,No,7.0,60
5,5,Sheila-kathryn,Calton,Female,56,Senior Editor,Manufacturing,Affluent Customer,N,Yes,8.0,44


##### CustomerAddress Dataset

In [298]:
customer_add.head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
3,4,0 Holy Cross Court,4211,QLD,Australia,9
4,5,17979 Del Mar Point,2448,New South Wales,Australia,4


In [299]:
titlehead(customer_add)

In [300]:
customer_add.head(2)

,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10


In [301]:
customer_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 1 to 3999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   object
 1   address             3999 non-null   object
 2   postcode            3999 non-null   object
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   object
dtypes: object(6)
memory usage: 218.7+ KB


In [302]:
customer_add[customer_add.duplicated()]

,customer_id,address,postcode,state,country,property_valuation


### Merge all customer information from tables into one table

In [303]:
transactions_demo = transactions.merge(customer_demo,on='customer_id',how='inner')
transactions_demo.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,...,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66
1,11065,1,2950,2017-10-16,False,Approved,Giant Bicycles,Standard,medium,medium,...,Anthony,Male,19,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,66


In [304]:
# merge customersdemographics & transactions with addresses dataset into a CTA datasets (customers,transactions,addresses)
CTA=transactions_demo.merge(customer_add,on='customer_id',how='inner')
CTA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19773 entries, 0 to 19772
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   transaction_id                       19773 non-null  object        
 1   product_id                           19773 non-null  object        
 2   customer_id                          19773 non-null  object        
 3   transaction_date                     19773 non-null  datetime64[ns]
 4   online_order                         19773 non-null  bool          
 5   order_status                         19773 non-null  object        
 6   brand                                19773 non-null  object        
 7   product_line                         19773 non-null  object        
 8   product_class                        19773 non-null  object        
 9   product_size                         19773 non-null  object        
 10  list_price

In [305]:
CTA.to_csv('CTA.csv')